In [27]:
import pandas as pd
import numpy as np
import joblib
import pymysql

In [39]:
db_name = "gis_system"
db_host = "localhost"
db_username = "root"
db_password = ""

try:
    conn = pymysql.connect(host = db_host,
                           port = int(3306),
                           user = "root",
                           password = db_password,
                           db = db_name)
except e:
    print (e)
if conn:
    print ("connection successful")
else:
    print ("error")

connection successful


In [29]:
model = joblib.load('model/ModelClassifier.pkl')

In [30]:
df_period = pd.read_csv('dataset/period3_psch_ht.csv')
df_crop_recom_range = pd.read_csv('dataset/crop_recommendation_range.csv')

In [31]:
df_period = df_period.rename(columns={
    'rata_rata_humi': 'humidity',
    'Curah_Hujan_Rata': 'rainfall',
    'rata_rata_temp': 'temperature'
})

In [32]:
features = df_period[['temperature', 'humidity', 'rainfall']].copy()

In [33]:
predictions = model.predict(features)

df_period['predicted_usable'] = predictions

In [34]:
data = pd.read_csv('dataset/forecast_location-yogyakarta.csv')

data['Waktu'] = pd.to_datetime(data['Waktu'])

filtered_data = data[data['Waktu'].dt.time == pd.to_datetime('06:00:00').time()]

filtered_data = filtered_data.drop(columns=['Suhu Min', 'Suhu Max', 'Kelembapan Min', 'Kelembapan Max'])

In [35]:
for index, row in filtered_data.iterrows():
    cursor = conn.cursor()
    sql = "SELECT * FROM forecast_weekly WHERE lat = %s AND lon = %s AND waktu = %s"
    cursor.execute(sql, (row['Latitude'], row['Longitude'], row['Waktu']))
    result = cursor.fetchall()
    if len(result) > 0:
        sql = "UPDATE forecast_weekly SET waktu = %s, kelembapan = %s, suhu = %s, cuaca = %s, arah_angin = %s, kecepatan_angin = %s, nama_daerah = %s, kabupaten_kota = %s, provinsi = %s WHERE lat = %s AND lon = %s"
        cursor.execute(sql, (row['Waktu'], row['Kelembapan'], row['Suhu'], row['Cuaca'], row['Arah Angin'], row['Kecepatan Angin'], row['Nama Daerah'], row['Kabupaten/Kota'], row['Provinsi'], row['Latitude'], row['Longitude']))
    else:
        sql = "INSERT INTO forecast_weekly (lat, lon, waktu, kelembapan, suhu, cuaca, arah_angin, kecepatan_angin, nama_daerah, kabupaten_kota, provinsi) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, (row['Latitude'], row['Longitude'], row['Waktu'], row['Kelembapan'], row['Suhu'], row['Cuaca'], row['Arah Angin'], row['Kecepatan Angin'], row['Nama Daerah'], row['Kabupaten/Kota'], row['Provinsi']))
    conn.commit()
    cursor.close()

for index, row in df_period.iterrows():
    cursor = conn.cursor()
    sql = "SELECT * FROM predictions WHERE lat = %s AND lon = %s"
    cursor.execute(sql, (row['LAT'], row['LON']))
    result = cursor.fetchall()
    if len(result) > 0:
        sql = "UPDATE predictions SET provinsi = %s, rainfall = %s, sifat_hujan_rata = %s, temperature = %s, humidity = %s, predicted = %s WHERE lat = %s AND lon = %s"
        cursor.execute(sql, (row['Id Prov'], row['rainfall'], row['Sifat_Hujan_Rata'], row['temperature'], row['humidity'], row['predicted_usable'], row['LAT'], row['LON']))
    else:
        sql = "INSERT INTO predictions (lat, lon, provinsi, rainfall, sifat_hujan_rata, temperature, humidity, predicted) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, (row['LAT'], row['LON'], row['Id Prov'], row['rainfall'], row['Sifat_Hujan_Rata'], row['temperature'], row['humidity'], row['predicted_usable']))
    conn.commit()
    cursor.close()
    
for index, row in df_crop_recom_range.iterrows():
    cursor = conn.cursor()
    sql = "SELECT * FROM crop_recom_range WHERE label = %s"
    cursor.execute(sql, (row['label']))
    result = cursor.fetchall()
    if len(result) > 0:
        sql = "UPDATE crop_recom_range SET min_range_temperature = %s, max_range_temperature = %s, min_range_humidity = %s, max_range_humidity = %s, min_range_rainfall = %s, max_range_rainfall = %s WHERE label = %s"
        cursor.execute(sql, (row['min_range_temperature'], row['max_range_temperature'], row['min_range_humidity'], row['max_range_humidity'], row['min_range_rainfall'], row['max_range_rainfall'], row['label']))
    else:
        sql = "INSERT INTO crop_recom_range (label, min_range_temperature, max_range_temperature, min_range_humidity, max_range_humidity, min_range_rainfall, max_range_rainfall) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, (row['label'], row['min_range_temperature'], row['max_range_temperature'], row['min_range_humidity'], row['max_range_humidity'], row['min_range_rainfall'], row['max_range_rainfall']))
    conn.commit()
    cursor.close()

df_period.to_csv('result/predicted_tanaman.csv', index=False)